In [ ]:
import cartopy.crs as ccrs
from cmocean import cm 
from dino import Experiment
from matplotlib import colors
from matplotlib import pyplot as plt
import numpy as np
import xarray as xr
import cftime as cft
import xnemogcm as xn
import xgcm

In [ ]:
path   = "/data/dkamm/nemo_output/DINO/"
dino_exp = Experiment(path, 'DensityRestoring/restart15')

In [ ]:
fig, axs = plt.subplots(1,1, figsize=(12,6))
dino_exp.get_ACC().plot(color='black')
axs.set_ylabel('ACC transport [Sv]')
plt.title('')
plt.xlabel('time [years]')
plt.legend()
plt.grid()

In [ ]:
T_star      = dino_exp.get_T_star()
S_star      = dino_exp.get_S_star()
rho_star    = dino_exp.get_rho_star()

In [ ]:
fig, axs = plt.subplots(1,3,figsize=(15,8), sharey=True)
T_star.isel(x_c=10, t_m=-1, y_c=slice(1,-1)).plot(color='midnightblue', y='gphit', ax=axs[0], label='December')
T_star.isel(x_c=10, t_m=-7, y_c=slice(1,-1)).plot(color='darkred', y='gphit', ax=axs[0], label='June')
S_star.isel(x_c=10, t_m=-1).plot(color='black', y='gphit', ax=axs[1])
rho_star.isel(x_c=10, t_m=-1, y_c=slice(1,-1)).plot(color='midnightblue', y='gphit', ax=axs[2], label='December')
rho_star.isel(x_c=10, t_m=-7, y_c=slice(1,-1)).plot(color='darkred', y='gphit', ax=axs[2], label='June')

axs[0].fill_betweenx(dino_exp.domain.gphiu.values[:,0], -2, 32, where=dino_exp.domain.umask.isel(x_f=0, z_c=0)==1, facecolor='grey', alpha=.2)
axs[0].margins(x=0, y=0)
axs[0].set_xlabel(r"$T^{*}$ [°C]")
axs[0].set_title('')
axs[0].grid()
axs[0].legend()
#
axs[1].fill_betweenx(dino_exp.domain.gphiu.values[:,0], 34., 36.5, where=dino_exp.domain.umask.isel(x_f=0, z_c=0)==1, facecolor='grey', alpha=.2)
axs[1].margins(x=0, y=0)
axs[1].set_xlabel(r"$S^{*}$ [psu]")
axs[1].set_title('')
axs[1].grid()
#
axs[2].fill_betweenx(dino_exp.domain.gphiu.values[:,0], 1022, 1028, where=dino_exp.domain.umask.isel(x_f=0, z_c=0)==1, facecolor='grey', alpha=.2)
axs[2].margins(x=0, y=0)
axs[2].set_xlabel(r"$\rho^{*}$ [kg / m3]")
axs[2].set_title('')
axs[2].grid()
axs[2].legend()

In [ ]:
moc =  dino_exp.get_MOC(dino_exp.data.voce + dino_exp.data.voce_eiv)

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(8,6))
a = (-moc).plot.contourf(x='y_f', y='rhop', cmap='RdBu_r', levels=36, add_colorbar=False)
cbar1 = fig.colorbar(a, ax=axs, label=r'$\psi$ [Sv]')
plt.xlabel('latitude [°N]')
axs.invert_yaxis()
plt.ylabel(r'$\sigma_{0}$ [ $kg$ / $m^3$ - 1000 ]')
plt.title('')

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(8,6))
a = dino_exp.data.rhop.isel(t_y=-1, x_c=10, y_c=slice(1,-1), z_c=slice(0,-1)).plot.contourf(x='gphit', y='gdept_0', cmap=cm.dense_r, levels=36, add_colorbar=False)
cbar1 = fig.colorbar(a, ax=axs, label=r'$\rho$ [kg/m^3]')
plt.xlabel('latitude [°N]')
axs.invert_yaxis()
plt.ylabel(r'$\sigma_{0}$ [ $kg$ / $m^3$ - 1000 ]')
plt.title('')

In [ ]:
from matplotlib.colors import LogNorm

soce = (dino_exp.data.isel(t_y=-1).where(dino_exp.domain.tmask == 1.)).soce.values.ravel()
toce = (dino_exp.data.isel(t_y=-1).where(dino_exp.domain.tmask == 1.)).toce.values.ravel()
vol  = (dino_exp.domain.e1t * dino_exp.domain.e2t * dino_exp.data.isel(t_y=-1).e3t).values.ravel()

smin, smax = 34, 36.5
tmin, tmax = -3, 28

s = np.linspace(smin, smax, 100)
t = np.linspace(tmin, tmax, 100)

Sg, Tg = np.meshgrid(s,t)

nml = dino_exp.namelist['nameos']

rho = (
    - nml['rn_a0'] * (1. + 0.5 * nml['rn_lambda1'] * ( Tg - 10.)) * ( Tg - 10.) 
    + nml['rn_b0'] * (1. - 0.5 * nml['rn_lambda2'] * ( Sg - 35.)) * ( Sg - 35.) 
    - nml['rn_nu'] * ( t - 10.) * ( Sg - 35.)
) + 1026

fig, ax = plt.subplots()
hb = ax.hexbin(soce, toce,
           C=vol, reduce_C_function=np.sum,
           extent=(34,36.5,-3,28), gridsize=50, bins='log',
           cmap=cm.matter)
plt.colorbar(hb)
cp = ax.contour(Sg,Tg, rho, levels=np.arange(1021, 1029, 0.5), linestyles='dashed', colors='black')
cl=plt.clabel(cp,fontsize=10,inline=True,fmt="%.1f")
ax.set_ylabel(r'Temperature ($^\circ$C)')
ax.set_xlabel("Salinity (g / kg)")

In [ ]:
mld_jul = dino_exp.data.mldr10_1.where(((dino_exp.data['t_m.year'] >= 530) & (dino_exp.data['t_m.month'] == 7)), drop=True).mean('t_m')
mld_feb = dino_exp.data.mldr10_1.where(((dino_exp.data['t_m.year'] >= 530) & (dino_exp.data['t_m.month'] == 2)), drop=True).mean('t_m')

mld_jul = mld_jul.assign_coords({'x_globe': mld_jul.glamt - 30})
mld_feb = mld_feb.assign_coords({'x_globe': mld_feb.glamt - 30})

In [ ]:
plt.figure(figsize=(10,10))
a = mld_feb.plot.contourf(
    y='gphit',
    x='x_globe',
    cmap=cm.deep_r,
    levels=30,
    subplot_kws=dict(projection=ccrs.Robinson()),

    transform=ccrs.PlateCarree(),
    #add_colorbar=False,
)
a.axes.gridlines(
    draw_labels=["x", "y", "geo"],
    ylocs=[-70, -45, -20, 0, 20, 45, 70],
    xlocs=[0]
)
plt.title('')
plt.grid('m')
plt.tight_layout()